<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li><li><span><a href="#Setup---initialize-ProquestHNPNewspaper" data-toc-modified-id="Setup---initialize-ProquestHNPNewspaper-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Setup - initialize ProquestHNPNewspaper</a></span><ul class="toc-item"><li><span><a href="#load-from-database" data-toc-modified-id="load-from-database-2.8.1"><span class="toc-item-num">2.8.1&nbsp;&nbsp;</span>load from database</a></span></li><li><span><a href="#set-up-manually" data-toc-modified-id="set-up-manually-2.8.2"><span class="toc-item-num">2.8.2&nbsp;&nbsp;</span>set up manually</a></span></li></ul></li></ul></li><li><span><a href="#Find-articles-to-be-loaded" data-toc-modified-id="Find-articles-to-be-loaded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be loaded</a></span><ul class="toc-item"><li><span><a href="#Uncompress-files" data-toc-modified-id="Uncompress-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Uncompress files</a></span></li><li><span><a href="#Work-with-uncompressed-files" data-toc-modified-id="Work-with-uncompressed-files-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Work with uncompressed files</a></span></li><li><span><a href="#parse-and-load-XML-files" data-toc-modified-id="parse-and-load-XML-files-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>parse and load XML files</a></span></li><li><span><a href="#build-list-of-all-ObjectTypes" data-toc-modified-id="build-list-of-all-ObjectTypes-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>build list of all ObjectTypes</a></span><ul class="toc-item"><li><span><a href="#Build-sorted-list-of-object-types" data-toc-modified-id="Build-sorted-list-of-object-types-3.4.1"><span class="toc-item-num">3.4.1&nbsp;&nbsp;</span>Build sorted list of object types</a></span></li></ul></li><li><span><a href="#map-files-to-types" data-toc-modified-id="map-files-to-types-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>map files to types</a></span><ul class="toc-item"><li><span><a href="#explore-all-known-object-types" data-toc-modified-id="explore-all-known-object-types-3.5.1"><span class="toc-item-num">3.5.1&nbsp;&nbsp;</span>explore all known object types</a></span></li><li><span><a href="#files-in-archive-BG_20171002210239_00001---1985" data-toc-modified-id="files-in-archive-BG_20171002210239_00001---1985-3.5.2"><span class="toc-item-num">3.5.2&nbsp;&nbsp;</span>files in archive BG_20171002210239_00001 - 1985</a></span></li><li><span><a href="#files-in-archive-BG_20171002210239_00001---1976" data-toc-modified-id="files-in-archive-BG_20171002210239_00001---1976-3.5.3"><span class="toc-item-num">3.5.3&nbsp;&nbsp;</span>files in archive BG_20171002210239_00001 - 1976</a></span></li></ul></li></ul></li><li><span><a href="#XML-analysis" data-toc-modified-id="XML-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>XML analysis</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
import datetime
import glob
import logging
import lxml
import os
import six
import xml
import xmltodict
import zipfile

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

What data are we looking at?

In [3]:
# paper identifier
paper_identifier = "BostonGlobe"
archive_identifier = "BG_20171002210239_00001"

# source
source_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data"
source_paper_path = "{}/{}".format( source_paper_folder, paper_identifier )

# uncompressed
uncompressed_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/uncompressed"
uncompressed_paper_path = "{}/{}".format( uncompressed_paper_folder, paper_identifier )

# make sure an identifier is set before you make a path here.
if ( ( archive_identifier is not None ) and ( archive_identifier != "" ) ):
    
    # identifier is set.
    source_archive_file = "{}.zip".format( archive_identifier )
    source_archive_path = "{}/{}".format( source_paper_path, source_archive_file )
    uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, archive_identifier )

#-- END check to see if archive_identifier present. --#

In [4]:
%pwd

'/home/jonathanmorgan/work/django/research/work/phd_work/data/article_loading/proquest_hnp/BostonGlobe'

In [5]:
# current working folder
current_working_folder = "/home/jonathanmorgan/work/django/research/work/phd_work/data/article_loading/proquest_hnp/{}".format( paper_identifier )
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime( "%Y-%m-%d-%H-%M-%S" )

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [6]:
logging_file_name = "{}/research-data_load-{}-{}.log.txt".format( current_working_folder, paper_identifier, current_date_string )
logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [7]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [8]:
%run $django_init_path

django initialized at 2019-08-19 17:43:05.649917


In [9]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

# context_text_proquest_hnp
from context_text_proquest_hnp.models import Proquest_HNP_Object_Type
from context_text_proquest_hnp.proquest_hnp_newspaper_helper import ProquestHNPNewspaperHelper

## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [10]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( "proquest_hnp-article-loading-{}".format( paper_identifier ) )
log_message = None

## Setup - initialize ProquestHNPNewspaper

- Back to [Table of Contents](#Table-of-Contents)

Create an initialize an instance of ProquestHNPNewspaper for this paper.

### load from database

- Back to [Table of Contents](#Table-of-Contents)

In [11]:
my_paper = ProquestHNPNewspaperHelper()
paper_instance = my_paper.initialize_from_database( paper_identifier )
my_paper.source_all_papers_folder = source_paper_folder
my_paper.destination_all_papers_folder = uncompressed_paper_folder

In [12]:
print( my_paper )
print( paper_instance )

BostonGlobe from 1872 to 1985 - source path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe - dest path: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe
1 - BostonGlobe - Boston Globe, The


### set up manually

- Back to [Table of Contents](#Table-of-Contents)

In [11]:
my_paper = ProquestHNPNewspaperHelper()
my_paper.paper_identifier = paper_identifier
my_paper.source_all_papers_folder = source_paper_folder
my_paper.source_paper_path = source_paper_path
my_paper.destination_all_papers_folder = uncompressed_paper_folder
my_paper.destination_paper_path = uncompressed_paper_path
my_paper.paper_start_year = 1872
my_paper.paper_end_year = 1985

my_newspaper = Newspaper.objects.get( id = 6 )
my_paper.newspaper = my_newspaper

If desired, add to database.

In [12]:
phnp_newspaper_instance = my_paper.create_PHNP_newspaper()

In [13]:
print( phnp_newspaper_instance )

1 - BostonGlobe - Boston Globe, The


# Find articles to be loaded

- Back to [Table of Contents](#Table-of-Contents)

Specify which folder of XML files should be loaded into system, then process all files within the folder.

The compressed archives from proquest_hnp just contain publication XML files, no containing folder.

To process:

- **uncompresed paper folder ( `<paper_folder>` )** - make a folder in `/mnt/hgfs/projects/phd/proquest_hnp/uncompressed` for the paper whose data you are working with, named the same as the paper's folder in `/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data`.

    - for example, for the Boston Globe, name it "`BostonGlobe`".

- **uncompressed archive folder ( `<archive_folder>` )** - inside a given paper's folder in uncompressed, for each archive file, create a folder named the same as the archive file, but with no ".zip" at the end.

    - For example, for the file "`BG_20171002210239_00001.zip`", make a folder named "`BG_20171002210239_00001`".
    - path should be "`<paper_folder>/<archive_name_no_zip>`.

- unzip the archive into this folder:

        unzip <path_to_zip> -d <archive_folder>



## Uncompress files

- Back to [Table of Contents](#Table-of-Contents)

See if the uncompressed paper folder exists.  If not, set flag and create it.

In [14]:
# create folder to hold the results of decompressing paper's zip files.
did_uncomp_paper_folder_exist = my_paper.make_dest_paper_folder()

EXISTS - Uncompressed paper folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe


For each *.zip file in the paper's source folder:

- parse file name from path returned by glob.
- parse the part before ".zip" from the file name.  This is referred to subsequently as the "archive identifier".
- check if folder named the same as the "archive identifier" is present.

    - If no:
    
        - create it.
        - then, uncompress the archive into it.
        
    - If yes:
    
        - output a message.  Don't want to uncompress if it was already uncompressed once.

In [15]:
# decompress the files
my_paper.uncompress_paper_zip_files()

==> zip file count: 474
----> processing file 1 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151210212722_00001.zip
==> file: BG_20151210212722_00001.zip
==> ID: BG_20151210212722_00001
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212722_00001
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212722_00001
------------------------------
----> processing file 2 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151210212824_00002.zip
==> file: BG_20151210212824_00002.zip
==> ID: BG_20151210212824_00002
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212824_00002
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212824_00002
------------------------------
----> processing file 3 of 474
==> path: /mnt/

==> file: BG_20151210223106_00006.zip
==> ID: BG_20151210223106_00006
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223106_00006
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223106_00006
------------------------------
----> processing file 71 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151210223207_00007.zip
==> file: BG_20151210223207_00007.zip
==> ID: BG_20151210223207_00007
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223207_00007
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223207_00007
------------------------------
----> processing file 72 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151210223208_00008.zip
==> file: BG_20151210223208_00008.zip
==> ID: BG_20151210223208_0000

==> file: BG_20151211000407_00010.zip
==> ID: BG_20151211000407_00010
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000407_00010
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000407_00010
------------------------------
----> processing file 147 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211001111_00001.zip
==> file: BG_20151211001111_00001.zip
==> ID: BG_20151211001111_00001
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211001111_00001
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211001111_00001
------------------------------
----> processing file 148 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211001213_00002.zip
==> file: BG_20151211001213_00002.zip
==> ID: BG_20151211001213_00

EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211020319_00010
------------------------------
----> processing file 229 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211020320_00011.zip
==> file: BG_20151211020320_00011.zip
==> ID: BG_20151211020320_00011
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211020320_00011
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211020320_00011
------------------------------
----> processing file 230 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211020421_00012.zip
==> file: BG_20151211020421_00012.zip
==> ID: BG_20151211020421_00012
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211020421_00012
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/proje

EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211032258_00012
------------------------------
----> processing file 297 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211032259_00013.zip
==> file: BG_20151211032259_00013.zip
==> ID: BG_20151211032259_00013
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211032259_00013
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211032259_00013
------------------------------
----> processing file 298 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211032301_00014.zip
==> file: BG_20151211032301_00014.zip
==> ID: BG_20151211032301_00014
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211032301_00014
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/proje

EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042322_00013
------------------------------
----> processing file 360 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211042424_00014.zip
==> file: BG_20151211042424_00014.zip
==> ID: BG_20151211042424_00014
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042424_00014
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042424_00014
------------------------------
----> processing file 361 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211042525_00015.zip
==> file: BG_20151211042525_00015.zip
==> ID: BG_20151211042525_00015
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042525_00015
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/proje

EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211052405_00018
------------------------------
----> processing file 440 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211052908_00001.zip
==> file: BG_20151211052908_00001.zip
==> ID: BG_20151211052908_00001
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211052908_00001
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211052908_00001
------------------------------
----> processing file 441 of 474
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/BostonGlobe/BG_20151211053011_00002.zip
==> file: BG_20151211053011_00002.zip
==> ID: BG_20151211053011_00002
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211053011_00002
EXISTS, so moving on - Uncompressed archive folder /mnt/hgfs/proje

## Work with uncompressed files

- Back to [Table of Contents](#Table-of-Contents)

Change working directories to the uncompressed paper path.

In [8]:
%cd $uncompressed_paper_path

/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe


In [9]:
%ls

BG_20151210212722_00001/  BG_20151211002934_00005/  BG_20151211034630_00005/
BG_20151210212824_00002/  BG_20151211003037_00006/  BG_20151211034731_00006/
BG_20151210212825_00003/  BG_20151211003138_00007/  BG_20151211034732_00007/
BG_20151210212926_00004/  BG_20151211003139_00008/  BG_20151211034833_00008/
BG_20151210212927_00005/  BG_20151211003240_00009/  BG_20151211034934_00009/
BG_20151210213028_00006/  BG_20151211004245_00001/  BG_20151211034935_00010/
BG_20151210213031_00007/  BG_20151211004347_00002/  BG_20151211035036_00011/
BG_20151210213132_00008/  BG_20151211004448_00003/  BG_20151211035037_00012/
BG_20151210213134_00009/  BG_20151211004549_00004/  BG_20151211035138_00013/
BG_20151210213235_00010/  BG_20151211004552_00005/  BG_20151211035239_00014/
BG_20151210213236_00011/  BG_20151211004653_00006/  BG_20151211035240_00015/
BG_20151210213337_00012/  BG_20151211004754_00007/  BG_20151211035341_00016/
BG_20151210213338_00013/  BG_20151211004755_00008/  BG_20151211035845_00001/

## parse and load XML files

- Back to [Table of Contents](#Table-of-Contents)

Load one of the files into memory and see what we can do with it.  Beautiful Soup?

Looks like the root element is "Record", then the high-level type of the article is "ObjectType".

ObjectType values:

- Advertisement
- ...

Good options for XML parser:

- `lxml.etree` - [https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python](https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python)
- `xmltodict` - [https://docs.python-guide.org/scenarios/xml/](https://docs.python-guide.org/scenarios/xml/)
- `beautifulsoup` using `lxml`

In [14]:
# loop over files in the current archive folder path.
object_type_to_count_map = my_paper.process_archive_object_types( uncompressed_archive_path )

Processing 5752 XML files in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001
----> XML file count: 5752
XML file count: 5752
Counters:
- Processed 5752 files
- No Record: 0
- No ObjectType: 0
- No ObjectType value: 0

ObjectType values and occurrence counts:
- A|d|v|e|r|t|i|s|e|m|e|n|t: 1902
- Article|Feature: 1792
- N|e|w|s: 53
- Commentary|Editorial: 36
- G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n: 488
- S|t|o|c|k| |Q|u|o|t|e: 185
- Advertisement|Classified Advertisement: 413
- E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c: 31
- Correspondence|Letter to the Editor: 119
- Front Matter|Table of Contents: 193
- O|b|i|t|u|a|r|y: 72
- F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y: 107
- I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h: 84
- Marriage Announcement|News: 6
- I|l|l|u|s|t|r|a|t|i|o|n: 91
- R|e|v|i|e|w: 133
- C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t: 30
- News|Legal Notice: 17


    Processing 5752 files in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001
    ----> XML file count: 5752

    Counters:
    - Processed 5752 files
    - No Record: 0
    - No ObjectType: 0
    - No ObjectType value: 0

    ObjectType values and occurrence counts:
    - A|d|v|e|r|t|i|s|e|m|e|n|t: 1902
    - Article|Feature: 1792
    - N|e|w|s: 53
    - Commentary|Editorial: 36
    - G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n: 488
    - S|t|o|c|k| |Q|u|o|t|e: 185
    - Advertisement|Classified Advertisement: 413
    - E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c: 31
    - Correspondence|Letter to the Editor: 119
    - Front Matter|Table of Contents: 193
    - O|b|i|t|u|a|r|y: 72
    - F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y: 107
    - I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h: 84
    - Marriage Announcement|News: 6
    - I|l|l|u|s|t|r|a|t|i|o|n: 91
    - R|e|v|i|e|w: 133
    - C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t: 30
    - News|Legal Notice: 17

## build list of all ObjectTypes

- Back to [Table of Contents](#Table-of-Contents)

Loop over all folders in the paper path.  For each folder, grab all files in the folder.  For each file, parse XML, then get the ObjectType value and if it isn't already in map of obect types to counts, add it.  Increment count.

From command line, in the uncompressed BostonGlobe folder:

    find . -type f -iname "*.xml" | wc -l

resulted in 11,374,500 articles.  That is quite a few.

In [14]:
xml_folder_list = glob.glob( "{}/*".format( uncompressed_paper_path ) )
print( "folder_list: {}".format( xml_folder_list ) )

folder_list: ['/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212722_00001', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212824_00002', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212825_00003', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212926_00004', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212927_00005', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213028_00006', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213031_00007', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213132_00008', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213134_00009', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213235_00010', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210213236_00011', '/mnt/hgfs/project

In [15]:
# build map of all object types for a paper to the overall counts of each
paper_object_type_to_count_map = my_paper.process_paper_object_types()

Processing 474 XML folders in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe
==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212722_00001 ( 1 of 474 ) @ 2019-08-15 05:19:18.880092
----> Processing complete @ 2019-08-15 05:20:43.997772 ( duration 0:01:25.117680 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212824_00002 ( 2 of 474 ) @ 2019-08-15 05:20:43.998197
----> Processing complete @ 2019-08-15 05:22:10.260852 ( duration 0:01:26.262655 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212825_00003 ( 3 of 474 ) @ 2019-08-15 05:22:10.261221
----> Processing complete @ 2019-08-15 05:24:01.944264 ( duration 0:01:51.683043 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210212926_00004 ( 4 of 474 ) @ 2019-08-15 05:24:01.944842
----> Processing complete @ 2019-08-15 05:25:

----> Processing complete @ 2019-08-15 06:07:59.185749 ( duration 0:00:57.845255 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210215814_00001 ( 36 of 474 ) @ 2019-08-15 06:07:59.187352
----> Processing complete @ 2019-08-15 06:09:28.186990 ( duration 0:01:28.999638 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210215915_00002 ( 37 of 474 ) @ 2019-08-15 06:09:28.187387
----> Processing complete @ 2019-08-15 06:10:50.446680 ( duration 0:01:22.259293 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210215916_00003 ( 38 of 474 ) @ 2019-08-15 06:10:50.447181
----> Processing complete @ 2019-08-15 06:12:11.687101 ( duration 0:01:21.239920 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210220019_00004 ( 39 of 474 ) @ 2019-08-15 06:12:11.687492
----> Processing complete @ 2019-08-15 06:13:32.

----> Processing complete @ 2019-08-15 06:56:35.560845 ( duration 0:01:20.725394 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223207_00007 ( 71 of 474 ) @ 2019-08-15 06:56:35.561932
----> Processing complete @ 2019-08-15 06:57:56.319330 ( duration 0:01:20.757398 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223208_00008 ( 72 of 474 ) @ 2019-08-15 06:57:56.319824
----> Processing complete @ 2019-08-15 06:59:16.748654 ( duration 0:01:20.428830 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223309_00009 ( 73 of 474 ) @ 2019-08-15 06:59:16.749065
----> Processing complete @ 2019-08-15 07:00:37.126564 ( duration 0:01:20.377499 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210223310_00010 ( 74 of 474 ) @ 2019-08-15 07:00:37.126958
----> Processing complete @ 2019-08-15 07:01:57.

----> Processing complete @ 2019-08-15 07:42:51.801512 ( duration 0:01:20.331786 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230656_00013 ( 106 of 474 ) @ 2019-08-15 07:42:51.801906
----> Processing complete @ 2019-08-15 07:43:44.044068 ( duration 0:00:52.242162 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210231201_00001 ( 107 of 474 ) @ 2019-08-15 07:43:44.044467
----> Processing complete @ 2019-08-15 07:45:04.860159 ( duration 0:01:20.815692 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210231303_00002 ( 108 of 474 ) @ 2019-08-15 07:45:04.861002
----> Processing complete @ 2019-08-15 07:46:26.516782 ( duration 0:01:21.655780 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210231404_00003 ( 109 of 474 ) @ 2019-08-15 07:46:26.517171
----> Processing complete @ 2019-08-15 07:47

----> Processing complete @ 2019-08-15 08:28:31.247459 ( duration 0:01:21.471114 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000059_00005 ( 141 of 474 ) @ 2019-08-15 08:28:31.247857
----> Processing complete @ 2019-08-15 08:29:51.599738 ( duration 0:01:20.351881 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000102_00006 ( 142 of 474 ) @ 2019-08-15 08:29:51.600455
----> Processing complete @ 2019-08-15 08:31:12.159933 ( duration 0:01:20.559478 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000203_00007 ( 143 of 474 ) @ 2019-08-15 08:31:12.160333
----> Processing complete @ 2019-08-15 08:32:31.698463 ( duration 0:01:19.538130 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211000305_00008 ( 144 of 474 ) @ 2019-08-15 08:32:31.698860
----> Processing complete @ 2019-08-15 08:33

...

----> Processing complete @ 2019-08-15 12:15:39.543165 ( duration 0:01:24.492501 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211034629_00004 ( 316 of 474 ) @ 2019-08-15 12:15:39.543569
----> Processing complete @ 2019-08-15 12:17:03.345281 ( duration 0:01:23.801712 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211034630_00005 ( 317 of 474 ) @ 2019-08-15 12:17:03.345676
----> Processing complete @ 2019-08-15 12:18:26.254933 ( duration 0:01:22.909257 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211034731_00006 ( 318 of 474 ) @ 2019-08-15 12:18:26.255374
----> Processing complete @ 2019-08-15 12:19:49.752169 ( duration 0:01:23.496795 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211034732_00007 ( 319 of 474 ) @ 2019-08-15 12:19:49.752636
----> Processing complete @ 2019-08-15 12:21

----> Processing complete @ 2019-08-15 13:03:35.388946 ( duration 0:01:28.510970 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211041813_00005 ( 351 of 474 ) @ 2019-08-15 13:03:35.389329
----> Processing complete @ 2019-08-15 13:04:58.513944 ( duration 0:01:23.124615 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211041914_00006 ( 352 of 474 ) @ 2019-08-15 13:04:58.514918
----> Processing complete @ 2019-08-15 13:06:21.948724 ( duration 0:01:23.433806 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042015_00007 ( 353 of 474 ) @ 2019-08-15 13:06:21.949473
----> Processing complete @ 2019-08-15 13:07:47.737203 ( duration 0:01:25.787730 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211042016_00008 ( 354 of 474 ) @ 2019-08-15 13:07:47.737608
----> Processing complete @ 2019-08-15 13:09

----> Processing complete @ 2019-08-15 13:52:11.507774 ( duration 0:01:23.149251 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211044857_00004 ( 386 of 474 ) @ 2019-08-15 13:52:11.508183
----> Processing complete @ 2019-08-15 13:53:36.216814 ( duration 0:01:24.708631 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211044858_00005 ( 387 of 474 ) @ 2019-08-15 13:53:36.217535
----> Processing complete @ 2019-08-15 13:55:10.639744 ( duration 0:01:34.422209 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211044900_00007 ( 388 of 474 ) @ 2019-08-15 13:55:10.640161
----> Processing complete @ 2019-08-15 13:56:32.818527 ( duration 0:01:22.178366 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211044959_00006 ( 389 of 474 ) @ 2019-08-15 13:56:32.818930
----> Processing complete @ 2019-08-15 13:57

----> Processing complete @ 2019-08-15 14:40:54.840381 ( duration 0:01:21.943371 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211051139_00021 ( 421 of 474 ) @ 2019-08-15 14:40:54.840800
----> Processing complete @ 2019-08-15 14:41:06.825625 ( duration 0:00:11.984825 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211051442_00001 ( 422 of 474 ) @ 2019-08-15 14:41:06.826318
----> Processing complete @ 2019-08-15 14:42:32.338421 ( duration 0:01:25.512103 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211051647_00002 ( 423 of 474 ) @ 2019-08-15 14:42:32.339161
----> Processing complete @ 2019-08-15 14:43:56.886408 ( duration 0:01:24.547247 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211051648_00003 ( 424 of 474 ) @ 2019-08-15 14:43:56.886861
----> Processing complete @ 2019-08-15 14:45

----> Processing complete @ 2019-08-15 15:27:12.338441 ( duration 0:00:25.674599 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211054032_00001 ( 456 of 474 ) @ 2019-08-15 15:27:12.338783
----> Processing complete @ 2019-08-15 15:28:37.213423 ( duration 0:01:24.874640 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211054133_00002 ( 457 of 474 ) @ 2019-08-15 15:28:37.214155
----> Processing complete @ 2019-08-15 15:30:02.613639 ( duration 0:01:25.399484 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211054235_00003 ( 458 of 474 ) @ 2019-08-15 15:30:02.614048
----> Processing complete @ 2019-08-15 15:31:28.565022 ( duration 0:01:25.950974 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151211054236_00004 ( 459 of 474 ) @ 2019-08-15 15:31:28.565400
----> Processing complete @ 2019-08-15 15:32

XML file count: 5752
Counters:
- Processed 5752 files
- No Record: 0
- No ObjectType: 0
- No ObjectType value: 0

ObjectType values and occurrence counts:
- A|d|v|e|r|t|i|s|e|m|e|n|t: 2114224
- Feature|Article: 5271887
- I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h: 249942
- O|b|i|t|u|a|r|y: 625143
- G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n: 1083164
- S|t|o|c|k| |Q|u|o|t|e: 202776
- N|e|w|s: 140274
- I|l|l|u|s|t|r|a|t|i|o|n: 106925
- F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y: 386421
- E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c: 78993
- Editorial|Commentary: 156342
- C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t: 68356
- Classified Advertisement|Advertisement: 291533
- R|e|v|i|e|w: 86889
- Table of Contents|Front Matter: 69798
- Letter to the Editor|Correspondence: 202071
- News|Legal Notice: 24053
- News|Marriage Announcement: 41314
- B|i|r|t|h| |N|o|t|i|c|e: 926
- News|Military/War News: 3
- U|n|d|e|f|i|n|e|d: 5
- Article|Feature: 137526
- Front Matter|Table of Contents: 11195
- Commentary|Editorial: 3386
- Marriage Announcement|News: 683
- Correspondence|Letter to the Editor: 7479
- Legal Notice|News: 1029
- Advertisement|Classified Advertisement: 12163


### Build sorted list of object types

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
# put the raw output from above in a list
raw_object_type_list = [ 'A|d|v|e|r|t|i|s|e|m|e|n|t: 2114224', 'Feature|Article: 5271887', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h: 249942', 'O|b|i|t|u|a|r|y: 625143', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n: 1083164', 'S|t|o|c|k| |Q|u|o|t|e: 202776', 'N|e|w|s: 140274', 'I|l|l|u|s|t|r|a|t|i|o|n: 106925', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y: 386421', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c: 78993', 'Editorial|Commentary: 156342', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t: 68356', 'Classified Advertisement|Advertisement: 291533', 'R|e|v|i|e|w: 86889', 'Table of Contents|Front Matter: 69798', 'Letter to the Editor|Correspondence: 202071', 'News|Legal Notice: 24053', 'News|Marriage Announcement: 41314', 'B|i|r|t|h| |N|o|t|i|c|e: 926', 'News|Military/War News: 3', 'U|n|d|e|f|i|n|e|d: 5', 'Article|Feature: 137526', 'Front Matter|Table of Contents: 11195', 'Commentary|Editorial: 3386', 'Marriage Announcement|News: 683', 'Correspondence|Letter to the Editor: 7479', 'Legal Notice|News: 1029', 'Advertisement|Classified Advertisement: 12163' ]

In [25]:
# output variable
master_object_type_list = None

# declare variables
#raw_object_type_list = None
raw_object_type = None
object_type_part_list = None
object_type_to_count_map = None
object_type_value = None
object_type_count_string = None
object_type_count = None

# loop
master_object_type_list = []
object_type_to_count_map = {}
for raw_object_type in raw_object_type_list:
    
    # split on colon
    object_type_part_list = raw_object_type.split( ":" )
    
    # object type value - take the first thing, strip off spaces, and add it to list.
    object_type_value = object_type_part_list[ 0 ]
    object_type_value = object_type_value.strip()

    # object type value count - item 2 (index 1)
    object_type_count_string = object_type_part_list[ 1 ]
    object_type_count_string = object_type_count_string.strip()
    object_type_count = int( object_type_count_string )
    
    # add value to list.
    if ( object_type_value not in master_object_type_list ):
        
        # add it.
        master_object_type_list.append( object_type_value )
        
    else:
        
        # error.
        print( "ERROR - object type value {} in list more than once.  Hmmm.".format( object_type_value ) )
        
    #-- END check to see if value already in list. --#
    
    # add count to map.
    if ( object_type_value not in object_type_to_count_map ):
        
        # add count.
        object_type_to_count_map[ object_type_value ] = object_type_count
        
    else:
        
        # error.
        print( "ERROR - object type value {} already has count in map.  Hmmm.".format( object_type_value ) )
        
    #-- END check to see if value already in list. --#
    
#-- END loop over raw object types --#

# sort the list of object types
master_object_type_list.sort()

print( master_object_type_list )

['Advertisement|Classified Advertisement', 'Article|Feature', 'A|d|v|e|r|t|i|s|e|m|e|n|t', 'B|i|r|t|h| |N|o|t|i|c|e', 'Classified Advertisement|Advertisement', 'Commentary|Editorial', 'Correspondence|Letter to the Editor', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t', 'Editorial|Commentary', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c', 'Feature|Article', 'Front Matter|Table of Contents', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n', 'I|l|l|u|s|t|r|a|t|i|o|n', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h', 'Legal Notice|News', 'Letter to the Editor|Correspondence', 'Marriage Announcement|News', 'News|Legal Notice', 'News|Marriage Announcement', 'News|Military/War News', 'N|e|w|s', 'O|b|i|t|u|a|r|y', 'R|e|v|i|e|w', 'S|t|o|c|k| |Q|u|o|t|e', 'Table of Contents|Front Matter', 'U|n|d|e|f|i|n|e|d']


    ['Advertisement|Classified Advertisement', 'Article|Feature', 'A|d|v|e|r|t|i|s|e|m|e|n|t', 'B|i|r|t|h| |N|o|t|i|c|e', 'Classified Advertisement|Advertisement', 'Commentary|Editorial', 'Correspondence|Letter to the Editor', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t', 'Editorial|Commentary', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c', 'Feature|Article', 'Front Matter|Table of Contents', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n', 'I|l|l|u|s|t|r|a|t|i|o|n', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h', 'Legal Notice|News', 'Letter to the Editor|Correspondence', 'Marriage Announcement|News', 'News|Legal Notice', 'News|Marriage Announcement', 'News|Military/War News', 'N|e|w|s', 'O|b|i|t|u|a|r|y', 'R|e|v|i|e|w', 'S|t|o|c|k| |Q|u|o|t|e', 'Table of Contents|Front Matter', 'U|n|d|e|f|i|n|e|d']


## map files to types

- Back to [Table of Contents](#Table-of-Contents)

Choose a directory, then loop over the files in the directory to build a map of types to lists of file names.

In [48]:
news_object_type_list = []
news_object_type_list.append( 'Article|Feature' )
news_object_type_list.append( 'Feature|Article' )
news_object_type_list.append( 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y' )

### explore all known object types

- Back to [Table of Contents](#Table-of-Contents)

Look at all known object types to see which contain actual news content.

In [16]:
# get list of all object types
master_object_type_list = my_paper.get_all_object_types()
print( "Object Types: {}".format( master_object_type_list ) )

Object Types: ['A|d|v|e|r|t|i|s|e|m|e|n|t', 'Advertisement|Classified Advertisement', 'Article|Feature', 'B|i|r|t|h| |N|o|t|i|c|e', 'Classified Advertisement|Advertisement', 'Commentary|Editorial', 'Correspondence|Letter to the Editor', 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t', 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c', 'Editorial|Commentary', 'Feature|Article', 'Front Matter|Table of Contents', 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y', 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n', 'I|l|l|u|s|t|r|a|t|i|o|n', 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h', 'Legal Notice|News', 'Letter to the Editor|Correspondence', 'Marriage Announcement|News', 'N|e|w|s', 'News|Legal Notice', 'News|Marriage Announcement', 'News|Military/War News', 'O|b|i|t|u|a|r|y', 'R|e|v|i|e|w', 'S|t|o|c|k| |Q|u|o|t|e', 'Table of Contents|Front Matter', 'Table Of Contents|Front Matter', 'U|n|d|e|f|i|n|e|d']


In [50]:
# directory to work in.
uncompressed_archive_folder = "BG_20171002210239_00001"
uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, uncompressed_archive_folder )
print( 'Uncompressed archive folder: {}'.format( uncompressed_archive_path ) )

Uncompressed archive folder: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001


In [14]:
# build map of file types to lists of files of that type in specified folder.
object_type_to_file_path_map = my_paper.map_archive_folder_files_to_types( uncompressed_archive_path )

Processing 5752 XML files in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001
----> XML file count: 5752

In map_archive_folder_files_to_types:
XML file count: 5752
Counters:
- Processed 5752 files
- No Record: 0
- No ObjectType: 0
- No ObjectType value: 0

ObjectType values and occurrence counts:
- Advertisement|Classified Advertisement - 413 files:
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821316533.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821332621.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821337723.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821337993.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821344353.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210

In [49]:
# which types do we want to preview?
#types_to_output = master_object_type_list
# NO - types_to_output = [ "Advertisement|Classified Advertisement" ]
# NO - types_to_output = [ "A|d|v|e|r|t|i|s|e|m|e|n|t" ]
# NO - types_to_output = [ 'Advertisement|Classified Advertisement' ]
# YES - types_to_output = [ 'Article|Feature' ]
# 0 - types_to_output = [ 'B|i|r|t|h| |N|o|t|i|c|e' ]
# 0 - types_to_output = [ 'Classified Advertisement|Advertisement' ]
# NO - types_to_output = [ 'Commentary|Editorial' ]
# NO - types_to_output = [ 'Correspondence|Letter to the Editor' ]
# NO - types_to_output = [ 'C|r|e|d|i|t|/|A|c|k|n|o|w|l|e|d|g|e|m|e|n|t' ]
# NO - types_to_output = [ 'E|d|i|t|o|r|i|a|l| |C|a|r|t|o|o|n|/|C|o|m|i|c' ]
# 0 - types_to_output = [ 'Editorial|Commentary' ]
# 0 - types_to_output = [ 'Feature|Article' ]
# NO - types_to_output = [ 'Front Matter|Table of Contents' ]
# YES - types_to_output = [ 'F|r|o|n|t| |P|a|g|e|/|C|o|v|e|r| |S|t|o|r|y' ]
# NO - furniture, listings - types_to_output = [ 'G|e|n|e|r|a|l| |I|n|f|o|r|m|a|t|i|o|n' ]
# NO - types_to_output = [ 'I|l|l|u|s|t|r|a|t|i|o|n' ]
# NO - types_to_output = [ 'I|m|a|g|e|/|P|h|o|t|o|g|r|a|p|h' ]
# 0 - types_to_output = [ 'Legal Notice|News' ]
# 0 - types_to_output = [ 'Letter to the Editor|Correspondence' ]
# NO - types_to_output = [ 'Marriage Announcement|News' ]
# NO - furniture, not actual articles - types_to_output = [ 'N|e|w|s' ]
# NO - types_to_output = [ 'News|Legal Notice' ]
# 0 - types_to_output = [ 'News|Marriage Announcement' ]
# 0 - types_to_output = [ 'News|Military/War News' ]
# NO - types_to_output = [ 'O|b|i|t|u|a|r|y' ]
# NO - types_to_output = [ 'R|e|v|i|e|w' ]
# NO - types_to_output = [ 'S|t|o|c|k| |Q|u|o|t|e' ]
# NO - types_to_output = [ 'Table of Contents|Front Matter' ]
# NO - types_to_output = [ 'Table Of Contents|Front Matter' ]
# NO - types_to_output = [ 'U|n|d|e|f|i|n|e|d' ]
types_to_output = news_object_type_list

# declare variables
xml_file_path_list = None
xml_file_path_count = None
xml_file_path_example_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_string = None

# loop over types
for object_type in types_to_output:
    
    # print type and count
    xml_file_path_list = object_type_to_file_path_map.get( object_type, [] )
    xml_file_path_count = len( xml_file_path_list )
    xml_file_path_example_list = xml_file_path_list[ : 10 ]
    print( "\n- {} - {} files:".format( object_type, xml_file_path_count ) )
    for xml_file_path in xml_file_path_example_list:
        
        print( "----> {}".format( xml_file_path ) )

        # try to parse the file
        with open( xml_file_path ) as xml_file:

            # parse XML
            xml_dict = xmltodict.parse( xml_file.read() )
            
        #-- END with open( xml_file_path ) as xml_file: --#
            
        # pretty-print
        xml_string = xmltodict.unparse( xml_dict, pretty = True )

        # output
        print( xml_string )
        
    #-- END loop over example file paths. --#
    
#-- END loop over object types. --#


- Article|Feature - 1792 files:
----> /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821304313.xml
<?xml version="1.0" encoding="utf-8"?>
<Record>
	<Version>Record_v1.0.xsd</Version>
	<RecordID>1821304313</RecordID>
	<DateTimeStamp>20171002164105</DateTimeStamp>
	<ActionCode>change</ActionCode>
	<RecordTitle>Learn, to prepare for life</RecordTitle>
	<Publication>
		<PublicationID>105359</PublicationID>
		<Title>Boston Globe (1960-1985)</Title>
		<Qualifier>Boston, Mass.</Qualifier>
	</Publication>
	<Publisher>Boston Globe Media Partners, LLC</Publisher>
	<AlphaPubDate>Oct 27, 1985</AlphaPubDate>
	<NumericPubDate>19851027</NumericPubDate>
	<SourceType>Historical Newspapers</SourceType>
	<ObjectType>Article</ObjectType>
	<ObjectType>Feature</ObjectType>
	<Contributor>
		<ContribRole>Author</ContribRole>
		<LastName>Wang</LastName>
		<FirstName>An</FirstName>
		<PersonName>An Wang</PersonName>
		<OriginalForm>An Wang</OriginalForm>
	</Contributor>
	

### files in archive BG_20171002210239_00001 - 1985

- Back to [Table of Contents](#Table-of-Contents)

Archive details:

- ID: 496
- Newspaper: 1 - BostonGlobe - Boston Globe, The
- archive_identifier: BG_20171002210239_00001
- min_date: Jan. 5, 1985
- max_date: Dec. 31, 1985
- path: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001

In [13]:
# directory to work in.
uncompressed_archive_folder = "BG_20171002210239_00001"
uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, uncompressed_archive_folder )
print( 'Uncompressed archive folder: {}'.format( uncompressed_archive_path ) )

Uncomressed archive folder: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001


In [14]:
# build map of file types to lists of files of that type in specified folder.
object_type_to_file_path_map = my_paper.map_archive_folder_files_to_types( uncompressed_archive_path )

Processing 5752 XML files in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001
----> XML file count: 5752

In map_archive_folder_files_to_types:
XML file count: 5752
Counters:
- Processed 5752 files
- No Record: 0
- No ObjectType: 0
- No ObjectType value: 0

ObjectType values and occurrence counts:
- Advertisement|Classified Advertisement - 413 files:
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821316533.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821332621.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821337723.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821337993.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821344353.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210

In [49]:
# which types do we want to preview?
types_to_output = news_object_type_list

# declare variables
xml_file_path_list = None
xml_file_path_count = None
xml_file_path_example_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_string = None

# loop over types
for object_type in types_to_output:
    
    # print type and count
    xml_file_path_list = object_type_to_file_path_map.get( object_type, [] )
    xml_file_path_count = len( xml_file_path_list )
    xml_file_path_example_list = xml_file_path_list[ : 10 ]
    print( "\n- {} - {} files:".format( object_type, xml_file_path_count ) )
    for xml_file_path in xml_file_path_example_list:
        
        print( "----> {}".format( xml_file_path ) )

        # try to parse the file
        with open( xml_file_path ) as xml_file:

            # parse XML
            xml_dict = xmltodict.parse( xml_file.read() )
            
        #-- END with open( xml_file_path ) as xml_file: --#
            
        # pretty-print
        xml_string = xmltodict.unparse( xml_dict, pretty = True )

        # output
        print( xml_string )
        
    #-- END loop over example file paths. --#
    
#-- END loop over object types. --#


- Article|Feature - 1792 files:
----> /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20171002210239_00001/1821304313.xml
<?xml version="1.0" encoding="utf-8"?>
<Record>
	<Version>Record_v1.0.xsd</Version>
	<RecordID>1821304313</RecordID>
	<DateTimeStamp>20171002164105</DateTimeStamp>
	<ActionCode>change</ActionCode>
	<RecordTitle>Learn, to prepare for life</RecordTitle>
	<Publication>
		<PublicationID>105359</PublicationID>
		<Title>Boston Globe (1960-1985)</Title>
		<Qualifier>Boston, Mass.</Qualifier>
	</Publication>
	<Publisher>Boston Globe Media Partners, LLC</Publisher>
	<AlphaPubDate>Oct 27, 1985</AlphaPubDate>
	<NumericPubDate>19851027</NumericPubDate>
	<SourceType>Historical Newspapers</SourceType>
	<ObjectType>Article</ObjectType>
	<ObjectType>Feature</ObjectType>
	<Contributor>
		<ContribRole>Author</ContribRole>
		<LastName>Wang</LastName>
		<FirstName>An</FirstName>
		<PersonName>An Wang</PersonName>
		<OriginalForm>An Wang</OriginalForm>
	</Contributor>
	

### files in archive BG_20171002210239_00001 - 1976

- Back to [Table of Contents](#Table-of-Contents)

Archive details:

- ID: 119
- Newspaper: 1 - BostonGlobe - Boston Globe, The
- archive_identifier: BG_20151210230044_00004
- min_date: 1960-04-25
- max_date: 1976-01-05
- path: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004

In [51]:
# directory to work in.
uncompressed_archive_folder = "BG_20151210230044_00004"
uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, uncompressed_archive_folder )
print( 'Uncompressed archive folder: {}'.format( uncompressed_archive_path ) )

Uncompressed archive folder: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004


In [52]:
# build map of file types to lists of files of that type in specified folder.
object_type_to_file_path_map = my_paper.map_archive_folder_files_to_types( uncompressed_archive_path )

Processing 25000 XML files in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004
----> XML file count: 25000

In map_archive_folder_files_to_types:
XML file count: 25000
Counters:
- Processed 25000 files
- No Record: 0
- No ObjectType: 0
- No ObjectType value: 0

ObjectType values and occurrence counts:
- A|d|v|e|r|t|i|s|e|m|e|n|t - 5064 files:
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367108498.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367113258.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367114973.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367138572.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367139373.xml
    - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367

In [53]:
# which types do we want to preview?
types_to_output = news_object_type_list

# declare variables
xml_file_path_list = None
xml_file_path_count = None
xml_file_path_example_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_string = None

# loop over types
for object_type in types_to_output:
    
    # print type and count
    xml_file_path_list = object_type_to_file_path_map.get( object_type, [] )
    xml_file_path_count = len( xml_file_path_list )
    xml_file_path_example_list = xml_file_path_list[ : 10 ]
    print( "\n- {} - {} files:".format( object_type, xml_file_path_count ) )
    for xml_file_path in xml_file_path_example_list:
        
        print( "----> {}".format( xml_file_path ) )

        # try to parse the file
        with open( xml_file_path ) as xml_file:

            # parse XML
            xml_dict = xmltodict.parse( xml_file.read() )
            
        #-- END with open( xml_file_path ) as xml_file: --#
            
        # pretty-print
        xml_string = xmltodict.unparse( xml_dict, pretty = True )

        # output
        print( xml_string )
        
    #-- END loop over example file paths. --#
    
#-- END loop over object types. --#


- Article|Feature - 0 files:

- Feature|Article - 9363 files:
----> /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/BostonGlobe/BG_20151210230044_00004/367091933.xml
<?xml version="1.0" encoding="utf-8"?>
<Record>
	<Version>Record_v1.0.xsd</Version>
	<RecordID>367091933</RecordID>
	<DateTimeStamp>20151210172823</DateTimeStamp>
	<ActionCode>change</ActionCode>
	<RecordTitle>Document Drafted to End Fear of Being Gypped</RecordTitle>
	<Publication>
		<PublicationID>105359</PublicationID>
		<Title>Boston Globe (1960-1984)</Title>
		<Qualifier>Boston, Mass.</Qualifier>
	</Publication>
	<Publisher>New York Times Company</Publisher>
	<AlphaPubDate>Nov 22, 1964</AlphaPubDate>
	<NumericPubDate>19641122</NumericPubDate>
	<SourceType>Historical Newspapers</SourceType>
	<ObjectType>Feature</ObjectType>
	<ObjectType>Article</ObjectType>
	<Contributor>
		<ContribRole>Author</ContribRole>
		<LastName>McPECK</LastName>
		<FirstName>ROBERT</FirstName>
		<PersonName>ROBERT McPECK</PersonName>
		<Origi

# XML analysis

- Back to [Table of Contents](#Table-of-Contents)

For analysis from across publications, see [proquest_hnp-article_loading.ipynb](../proquest_hnp-article_loading.ipynb)

# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- figure out which ObjectTypes to explore, pick a folder and just eyeball a few, to see what they look like.
- // map publication date ranges to files (for each folder, capture max and min pub date - element: "`NumericPubDate`").
- / make a little `proquest_hnp` django app to hold metadata about the publications.  Build on Context_Text:

    - // models:

        - papers in context_text.
        - object types for each paper.

            - pipe-delimited value.
            - display value
            - ?

        - folders for each paper.

            - folder path
            - min pub date
            - max pub_date

        - paths to files in each folder?
        - 
        
    - // code:
    
        - class for dealing with files and folders - migrate the code from jupyter notebooks here, instance variables for folder settings, logging, etc.
        
    - / admin:
    
        - // make custom admins for everything so I can control what is on the list page, make lookups for foreign keys, etc. (need to have counts on the list pages, etc.)
        - // add object type inlines for papers and archives so you can see related object types.
        - add paper inline to object types, so you can see which papers use each object type?
        - ...?
        
- // make the `summarize_archive_files` method also create Archive records in django if none exist.
- // run summarize on all Boston Globe articles, store the results, so we can start to look at article quality in different eras.